# Magnetomotive force along a line

This example shows how to use PyAEDT to calculate
the magnetomotive force along a line that changes position.
It shows how to leverage the PyAEDT advanced fields calculator
to insert a custom formula, which in this case is the integral
of the H field along a line.
The example shows two options to achieve the intent.
The first one creates many lines as to simulate a contour that changes position.
The integral of the H field is computed for each line.
The second option creates one parametric polyline and then uses a parametric sweep to change its position.
The integral of the H field is computed for each position.

Keywords: **Maxwell 2D**, **magnetomotive force**.

## Perform imports and define constants

Perform required imports.

In [1]:
import tempfile
import time

import ansys.aedt.core
from ansys.aedt.core.examples.downloads import download_file

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Import project

Download the files required to run this example to the temporary working folder.

In [4]:
project_path = download_file(
    source="maxwell_magnetic_force",
    name="Maxwell_Magnetic_Force.aedt",
    local_path=temp_folder.name,
)

## Initialize and launch Maxwell 2D

Initialize and launch Maxwell 2D, providing the version and the path of the project.

In [5]:
m2d = ansys.aedt.core.Maxwell2d(
    version=AEDT_VERSION,
    non_graphical=NG_MODE,
    project=project_path,
    new_desktop=True,
    design="Maxwell2DDesign1",
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpntcs104m.ansys\maxwell_magnetic_force\Maxwell_Magnetic_Force.aedt.


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


...

PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.


PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project Maxwell_Magnetic_Force has been opened.


PyAEDT INFO: Active Design set to Maxwell2DDesign1


PyAEDT INFO: Aedt Objects correctly read


# First option

## Create a polyline

Create a polyline, specifying its ends.

In [6]:
poly = m2d.modeler.create_polyline(points=[[10, -10, 0], [10, 10, 0]], name="polyline")

PyAEDT INFO: Modeler2D class has been initialized!


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


Duplicate the polyline along a vector.

In [7]:
polys = [poly.name]
polys.extend(poly.duplicate_along_line(vector=[-0.5, 0, 0], clones=10))

## Compute magnetomotive force along each line

Create and add a new formula to add in the PyAEDT advanced fields calculator.
Create the fields report object and get field data.
Create a data table report for the H field along each line and export it to a .csv file.

In [8]:
my_expression = {
    "name": None,
    "description": "Magnetomotive force along a line",
    "design_type": ["Maxwell 2D", "Maxwell 3D"],
    "fields_type": ["Fields"],
    "primary_sweep": "distance",
    "assignment": None,
    "assignment_type": ["Line"],
    "operations": [
        "Fundamental_Quantity('H')",
        "Operation('Tangent')",
        "Operation('Dot')",
        "EnterLine('assignment')",
        "Operation('LineValue')",
        "Operation('Integrate')",
    ],
    "report": ["Data Table"],
}

In [9]:
quantities = []
for p in polys:
    quantity = "H_field_{}".format(p)
    quantities.append(quantity)
    my_expression["name"] = quantity
    my_expression["assignment"] = quantity
    m2d.post.fields_calculator.add_expression(my_expression, p)
    report = m2d.post.create_report(
        expressions=quantity,
        report_category="Fields",
        plot_type="Data Table",
        plot_name=quantity
    )

PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


PyAEDT WARNING: No context provided for Fields. Assigned object: None


PyAEDT WARNING: No context provided for Fields. Assigned object: None


...

PyAEDT WARNING: No context provided for Fields. Assigned object: None


PyAEDT WARNING: No context provided for Fields. Assigned object: None


PyAEDT WARNING: No context provided for Fields. Assigned object: None


PyAEDT WARNING: No context provided for Fields. Assigned object: None


PyAEDT WARNING: No context provided for Fields. Assigned object: None


# Second option

## Create a design variable

Parametrize the polyline x position.

In [10]:
m2d["xl"] = "10mm"

## Create polyline

Create a parametrized polyline, specifying its ends.

In [11]:
poly = m2d.modeler.create_polyline(
    points=[["xl", -10, 0], ["xl", 10, 0]], name="polyline_sweep"
)

## Add parametric sweep

Add a parametric sweep where the parameter to sweep is ``xl``.
Create a linear step sweep from ``10mm`` to ``15mm`` every ``1mm`` step.

In [12]:
param_sweep = m2d.parametrics.add(
    variable="xl",
    start_point="10mm",
    end_point="15mm",
    step=1,
    variation_type="LinearStep",
)

## Compute magnetomotive force along the line

Create and add a new formula to add in the PyAEDT advanced fields calculator.

In [13]:
quantity_sweep = "H_field_{}".format(poly.name)
my_expression["name"] = quantity_sweep
my_expression["assignment"] = poly.name
m2d.post.fields_calculator.add_expression(my_expression, poly.name)

'H_field_polyline_sweep'

## Add parametric sweep calculation specifying the quantity (H) and save fields.

In [14]:
param_sweep.add_calculation(calculation=quantity_sweep, report_type="Fields", ranges={})
param_sweep.props["ProdOptiSetupDataV2"]["SaveFields"] = True

## Create data table report

Create a data table report to display H for each polyline position.

In [15]:
report_sweep = m2d.post.create_report(
    expressions=quantity_sweep,
    report_category="Fields",
    plot_type="Data Table",
    plot_name=quantity_sweep,
    primary_sweep_variable="xl",
    variations={"xl": "All"},
)

PyAEDT WARNING: No context provided for Fields. Assigned object: None


## Analyze parametric sweep

In [16]:
param_sweep.analyze(cores=NUM_CORES)

PyAEDT INFO: Project Maxwell_Magnetic_Force Saved correctly


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


PyAEDT INFO: Solving Optimetrics


PyAEDT INFO: Design setup Parametric_NCJE0D solved correctly in 0.0h 0.0m 31.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


True

## Export results

Export results in a .csv file for the parametric sweep analysis (second option).

In [17]:
m2d.post.export_report_to_csv(
    project_dir=temp_folder.name,
    plot_name=quantity_sweep,
)

'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_sweep.csv'

Export results in a .csv file for each polyline (first option).

In [18]:
[
    m2d.post.export_report_to_csv(
        project_dir=temp_folder.name,
        plot_name=q,
    )
    for q in quantities
]

['C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_1.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_2.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_3.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_4.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_5.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_6.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_7.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_8.csv',
 'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpntcs104m.ansys\\H_field_polyline_9.csv']

## Release AEDT

In [19]:
m2d.save_project()
m2d.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project Maxwell_Magnetic_Force Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [20]:
temp_folder.cleanup()